In [1]:
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load datasets
eng_sarc_dataset = load_dataset(path='raquiba/Sarcasm_News_Headline', split='train')
ar_sarc_dataset = load_dataset(path='ar_sarcasm', split='train', ignore_verifications=True)
kor_sarc_dataset = load_dataset(path='kor_sarcasm', split='train')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/raquiba--Sarcasm_News_Headline-2c0b20e2ac20e61f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2110 [00:00<?, ? examples/s]

Dataset ar_sarcasm downloaded and prepared to /root/.cache/huggingface/datasets/ar_sarcasm/default/1.0.0/946b5574cab73f8afb77406014d21a41f3d73d0d1922b8a675fa7449190b9753. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/301 [00:00<?, ? examples/s]

Dataset kor_sarcasm downloaded and prepared to /root/.cache/huggingface/datasets/kor_sarcasm/default/1.1.0/00d38c200d4d563ed94efb9ff4ca119ded94fe3cdf1e381ed95274de0a9d59f0. Subsequent calls will reuse this data.


In [3]:
### process datasets
eng_sarc_dataset = eng_sarc_dataset.remove_columns(column_names=['article_link'])
eng_sarc_dataset = eng_sarc_dataset.rename_columns({"is_sarcastic":"label"})

ar_sarc_dataset = ar_sarc_dataset.remove_columns(column_names=['dialect', 'sentiment', 'original_sentiment', 'source'])
ar_sarc_dataset = ar_sarc_dataset.rename_columns({"tweet":"headline", "sarcasm":"label"})

kor_sarc_dataset = kor_sarc_dataset.rename_columns({"tokens":"headline"})

In [4]:
model_ckpt = 'distilbert-base-uncased'

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [6]:
# tokenize datasets using distilbert pretrained tokenizer
encoded_eng_dataset = eng_sarc_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512, padding="max_length", truncation=True), batched=True)
encoded_ar_dataset = ar_sarc_dataset.map(lambda sample: tokenizer(sample['headline'],  max_length=512, padding="max_length", truncation=True), batched=True)
encoded_kor_dataset = kor_sarc_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512,  padding="max_length", truncation=True), batched=True)

  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [7]:
from transformers import AutoModelForSequenceClassification
import torch 

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.we

In [9]:
from transformers import TrainingArguments, Trainer

In [10]:
model_name = f"{model_ckpt}-multiling-sarcasm"
training_args = TrainingArguments(output_dir=model_name,
                                 num_train_epochs=10,
                                 learning_rate=1e-5,
                                 per_device_train_batch_size=32,
                                 per_device_eval_batch_size=32,
                                 evaluation_strategy='epoch',
                                 weight_decay=0.01,
                                 log_level='error',
                                 push_to_hub=False,
                                 report_to='none')

In [11]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np 

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions)
    accuracy = accuracy_score(labels, predictions)
    return {"f1": f1, "acc": accuracy}

In [12]:
encoded_eng_dataset_rdy = encoded_eng_dataset.train_test_split(test_size=0.25)

In [13]:
trainer = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_eng_dataset_rdy['train'],
                 eval_dataset=encoded_eng_dataset_rdy['test'])
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.238956,0.897513,0.904403
2,0.309800,0.210236,0.909583,0.916003
3,0.162700,0.201424,0.922339,0.925367
4,0.162700,0.219531,0.913744,0.919916
5,0.098800,0.236132,0.917305,0.922991
6,0.064600,0.268661,0.916001,0.922152
7,0.064600,0.281859,0.920490,0.924668
8,0.040800,0.308471,0.917228,0.922711
9,0.029900,0.311543,0.923191,0.927463
10,0.029900,0.310941,0.923327,0.927603


TrainOutput(global_step=3360, training_loss=0.10811667782919747, metrics={'train_runtime': 5904.0846, 'train_samples_per_second': 36.354, 'train_steps_per_second': 0.569, 'total_flos': 2.843280244752384e+16, 'train_loss': 0.10811667782919747, 'epoch': 10.0})

In [14]:
encoded_ar_dataset_rdy = encoded_ar_dataset.train_test_split(test_size=0.25)

In [15]:
trainer_ar = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_ar_dataset_rdy['train'],
                 eval_dataset=encoded_ar_dataset_rdy['test'])
trainer_ar.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.414819,0.000000,0.839810
2,No log,0.402305,0.011765,0.840758
3,No log,0.422430,0.017493,0.840284
4,No log,0.404054,0.104987,0.838389
5,No log,0.438206,0.086957,0.840758
6,0.382600,0.413949,0.222222,0.837441
7,0.382600,0.422134,0.173913,0.837915
8,0.382600,0.432207,0.179245,0.835071
9,0.382600,0.435356,0.201794,0.831280
10,0.382600,0.436964,0.199546,0.832701


TrainOutput(global_step=990, training_loss=0.3473057448261916, metrics={'train_runtime': 1762.4219, 'train_samples_per_second': 35.899, 'train_steps_per_second': 0.562, 'total_flos': 8381212312965120.0, 'train_loss': 0.3473057448261916, 'epoch': 10.0})

In [16]:
encoded_kor_dataset_rdy = encoded_kor_dataset.train_test_split(test_size=0.25)